<a href="https://colab.research.google.com/github/mpbeck/ANS-128-GP-The-Explorers-/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Change runtime to R
system("git clone https://github.com/mpbeck/ANS-128-GP-The-Explorers-.git")

In [15]:
library(tidyverse)
data <- read.csv("/content/ANS-128-GP-The-Explorers-/Avian Influenza (HPAI).csv")
class(data)

[1] "data.frame"

In [14]:
# Data cleaning & regression analysis
#summary(data$target_H5_HPAI)
data <- data %>%
  mutate(
    Common_Name = str_replace_all(Common_Name, " ", "_")
    Year = as.factor(Year),
    Month = as.factor(Month),
    Day = as.factor(Day),
    State = as.factor(State),
    County = as.factor(County),
    Locality = as.factor(Locality),
    AvianFluStatus = factor(AvianFluStatus, levels = c(0,1), labels = c("No","Yes"))
  )
length(unique(data$Common_Name))

[1] 410